In [ ]:
load_ext autoreload

In [ ]:
autoreload 2

In [ ]:
import numpy as np
import pyphi
import random
import makegridslibrary as me
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
def make_cmgrid_withbackground(N,d):
    C=np.zeros((2*N,2*N),int)
    C[0:N,0:N]=me.make_cmgrid(N,d)
    #add the ``structured'' background conditions
    for i in range(N):
        C[N+i,i]=1
        #C[N+i,(i+1)%N]=1

    return C

def make_random_withbackground(N,d):
    R=np.zeros((2*N,2*N),int)
    R[0:N,0:N]=me.make_random(N,d)
    #add the ``structured'' background conditions
    for i in range(N):
        R[N+i,i]=1
        #R[N+i,(i+1)%N]=1

    return R

def make_tpm_withbackground(N,cm,threshh): #pick thresh==0 so it calculate the thresh 
    #assumes that the background nodes are at the end, so cm dim is 2*N,2*N
    #state by node
    if threshh>0:
        thresh=threshh
    else:
        thresh=int((sum(cm)+1)/2)
    N2=2*N
    M = 2**N2
    tpm = np.zeros([M, N2])
    for i in range(M):
        state = pyphi.convert.loli_index2state(i, N2)
        for node in range(N):
            tpm[i, node] = np.dot(cm.transpose()[node], state) >= thresh
            
            
        # TODO LEO Not sur I understand why...
        for node2 in range(N):
            tpm[i,node2+N]=0.5
            
    return tpm


In [ ]:
# cm whole conn matrix including backgroun, N is the nr of nodes in the ``main complex'', state as initial cond

def running_behavior(cm,N,iterations,state,threshh,do_plot=True,do_print=True): #pick threshh ==0 to calculate it

    lcm=len(cm)
    scm=sum(cm)

    if threshh > 0:
        thresh = threshh
    else:
        thresh = (scm + 1) / 2

    fixbackground=state[N:lcm]
    statenew=np.zeros((lcm,iterations),int)
    statenew[:,0]=state
    for i in range(iterations-1):
            statenew[:,i+1] = np.dot(cm.transpose(), statenew[:,i]) >= thresh
            statenew[N:lcm,i+1]=fixbackground

            #print('this is the new state\n', statenew[:,i+1])

    statenew1=statenew.transpose()        
    statenew2=statenew1[::-1]    

    if do_plot:
        fig = plt.figure()

        ax = fig.add_subplot(1,2,1)
        ax.set_aspect('equal')
        plt.imshow(statenew2, interpolation='nearest', cmap=plt.cm.PuBu)
        plt.colorbar()

        ax = fig.add_subplot(1,2,2)
        ax.set_aspect('equal')
        plt.imshow(statenew2[:,0:N] - statenew2[:,N:(2*N)], interpolation='nearest', cmap=plt.cm.bwr)
        plt.colorbar()

        plt.show()
        
    distance = np.sum(np.abs(statenew[0:N,i+1]-fixbackground))
    
    if do_print:
        print('Distance: %d' % distance)
        
    return distance

    # return statenew



In [ ]:
def difference_of_matching(cm,N,iterations,state,threshh): #threshh==0 calculates the thresh as maj 
    lcm=len(cm)
    scm=sum(cm)
    if threshh>0:
        thresh=threshh
    else:
        thresh=(scm+1)/2
        
    fixbackground=state[N:lcm]
    statenew=np.zeros((lcm,iterations),int)
    statenew[:,0]=state
    dist=np.zeros(iterations,int)
    for i in range(iterations-1):
            statenew[:,i+1] = np.dot(cm.transpose(), statenew[:,i]) >= thresh
            statenew[N:lcm,i+1]=fixbackground
            distance=statenew[0:N,i+1]-fixbackground
            dist[i+1]=sum(abs(distance))
            #print('this is the new state\n', statenew[:,i+1])
            
    finaldist=dist[iterations-1]
    
    return finaldist

In [ ]:
def pick_random_state(N,nrfiring,nrfiringbackground):
    vv=random.sample(range(N),nrfiring)
    bgrvv=np.asarray(random.sample(range(N),nrfiringbackground))
    bgrvv=bgrvv+N
    bgrvv=bgrvv
    vvar=np.zeros(2*N,int)
    vvar[vv]=1
    vvar[bgrvv]=1
    background=vvar[N:]
    
    return vvar,background


In [ ]:
C_BGR_N9=make_cmgrid_withbackground(9,3)
C_BGR_N9

In [ ]:
C_BGR_N5=make_cmgrid_withbackground(5,3)
C_BGR_N5

In [ ]:
R_BGR_N5=make_random_withbackground(5,3)
R_BGR_N5

In [ ]:
T_BGR_N9=make_tpm_withbackground(9,C_BGR_N9,3)

In [ ]:
# C_N4 = me.make_cmgrid(4,3)
# print(C_N4)
# print(me.make_tpm(4, C_N4, 2))

T_BGR_N4=make_tpm_withbackground(4,C_BGR_N4,2)
T_BGR_N4

# fig = plt.figure()

# ax = fig.add_subplot(1,1,1)
# ax.set_aspect('equal')
# plt.imshow(T_BGR_N4, interpolation='nearest', cmap=plt.cm.PuBu)
# plt.colorbar()

# plt.show()

In [ ]:
#np.get_printoptions()
np.set_printoptions(edgeitems=3)

In [ ]:
running_behavior(C_BGR_N9,9,10,np.array([1,1,1,0,0,0,0,1,1,
                                         1,1,1,0,0,0,1,1,1]),3)

In [ ]:
# fixed background: grid or random graph

# Average distance for random initial states

# cm = C_BGR_N4
# N = 4
# thresh = 2
# initial_state = np.array([1,0,1,0,
#                          1,1,0,0])

# cm = C_BGR_N9
# N = int(cm.shape[0]/2)
# thresh = 3
# # intial state that lead to matching (very similar) : 1,1,1,0,0,0,0,1,1
# initial_state = np.array([1,1,1,0,0,0,0,1,1,
#                           1,1,0,0,0,0,1,1,1])

# cm = R_BGR_N5
cm = C_BGR_N5
N = int(cm.shape[0]/2)
thresh = 2
# intial state that lead to matching (very similar) : 1,1,1,0,0,0,0,1,1
initial_state = np.array([0,0,0,0,0,
                          0,0,1,1,0])

all_initial_conditions = itertools.product((0, 1), repeat=N)

iterations = 10
nsim = 2**N
distances = np.zeros(nsim)
good_cond = list()
isim = 0
# nsim = 10000
# for isim in range(nsim):
for this_state in all_initial_conditions:
    
    # this_state = np.random.randint(0, 2, N)

#     this_state = initial_state[0:N]
#     np.random.shuffle(this_state)

    state = initial_state
    state[0:N] = this_state
    
    if nsim > 1000:
        do_plot = False
    else:
        do_plot = True

    distances[isim] = running_behavior(cm,N,iterations,state,thresh, do_plot=do_plot, do_print=False)
   
    if distances[isim] == 0:
        good_cond.append(this_state)

    isim +=1
        
# print("Mean distance: %f (chance %f)" % (np.mean(1*(distances==0)), 1/(sum(initial_state[0:N])*2**(N-sum(initial_state[0:N])))))

# there must be a better way of doing that, but it does the trick

# from itertools import permutations
# perms = list(set(permutations(initial_state[N:])))

perms = list(all_initial_conditions)

chance_level = 1./isim

print("Mean distance: %f (chance %f)" % (np.mean(1*(distances==0)), chance_level))
print("N good %d" % len(good_cond))

good_cond


In [ ]:
# fixed initial start: structured and random background conditions

# Average distance for random initial states

# cm = C_BGR_N4
# N = 4
# thresh = 2
# initial_state = np.array([1,0,1,0,
#                          1,1,0,0])

# cm = C_BGR_N9
# N = int(cm.shape[0]/2)
# thresh = 3
# # intial state that lead to matching (very similar) : 1,1,1,0,0,0,0,1,1
# initial_state = np.array([1,1,1,0,0,0,0,1,1,
#                           1,1,0,0,0,0,1,1,1])

cm = R_BGR_N5
# cm = C_BGR_N5
N = int(cm.shape[0]/2)
thresh = 2
# intial state that lead to matching (very similar) : 1,1,1,0,0,0,0,1,1
initial_state = np.array([0,1,0,1,1,
                          0,0,1,1,0])

all_initial_conditions = itertools.product((0, 1), repeat=N)

iterations = 10
nsim = 2**N
distances = np.zeros(nsim)
good_cond = list()
isim = 0
# nsim = 10000
# for isim in range(nsim):
for this_state in all_initial_conditions:
    
    # this_state = np.random.randint(0, 2, N)

#     this_state = initial_state[0:N]
#     np.random.shuffle(this_state)

    state = initial_state
#     state[0:N] = this_state
    state[N:] = this_state
    
    if nsim > 10:
        do_plot = False
    else:
        do_plot = True

    distances[isim] = running_behavior(cm,N,iterations,state,thresh, do_plot=do_plot, do_print=False)
   
    if distances[isim] == 0:
        good_cond.append(this_state)

    isim +=1
        
# print("Mean distance: %f (chance %f)" % (np.mean(1*(distances==0)), 1/(sum(initial_state[0:N])*2**(N-sum(initial_state[0:N])))))

# there must be a better way of doing that, but it does the trick

# from itertools import permutations
# perms = list(set(permutations(initial_state[N:])))

perms = list(all_initial_conditions)

chance_level = 1./isim

print("Mean distance: %f (chance %f)" % (np.mean(1*(distances==0)), chance_level))
print("N good %d" % len(good_cond))

good_cond

In [ ]:
#test, for convervence using actual thresh
M=7
l=0.5
N=9
cm=C_BGR_N9
it=10
for i in range(M):
    nrfiring=int(l*N)
    nrfiringbackground=nrfiring
    [state,background]=pick_random_state(N,nrfiring,nrfiringbackground)
    running_behavior(cm,N,it,state,0)

In [ ]:
state=np.array([1,1,1,0,0,0,0,1,1,1,1,0,0,1,1,0,0,0])
difference_of_matching(C_BGR_N9,9,10,state,0)

In [ ]:
#Larissa mentioned, we should keep the thresh as if there would be no background conditions (added that in running_behavior)

In [ ]:
#compare to actual thresh
M=7
l=0.5
N=9
cm=R
it=10
print('this is the random matrix I look at\n', cm)
for i in range(M):
    nrfiring=int(l*N)
    nrfiringbackground=nrfiring
    [state,background]=pick_random_state(N,nrfiring,nrfiringbackground)
    running_behavior(cm,N,it,state,0)
    d=difference_of_matching(cm,N,it,state,0)
    print(d)

What we see is: 
- both networks converge to a particular state with activity